## 1) Import Modules required throughout

In [1]:
import requests
import time
import zipfile
import io
from pprint import pprint
import json

## 2) Set up your credentials (britbox, your token)

In [2]:
mytoken='ExTDN3nBP2Q8vVVnUJeMsjUCbyuWPyEDxblfUqCW'
org_id='britboxresearch'

In [3]:
accountinfo=Get_Account_Info(mytoken,org_id)
accountinfo['result']['stats']['responseCounts']['auditable']

In [ ]:
survey_directory=Get_list_of_surveys(mytoken)
survey_dict={i['name']:i['id'] for i in survey_directory if i['name'][-14:]=='Leavers survey'}

In [ ]:
survey_dict
surveyid=survey_dict['Content - Leavers survey']

In [ ]:
survey_details=Get_Survey_Info(mytoken,surveyid)

In [ ]:
# Get Survey API
survey_details

# Download survey from API

In [ ]:

####### Start the survey download
url = "https://eu.qualtrics.com/API/v3/surveys/{}/export-responses".format(surveyid)
headers = {"content-type": "application/json",
    "X-API-TOKEN": mytoken,
    'Accept-Charset': 'UTF-8',
    'compress':'true'
    }
response = requests.request("POST", url, headers=headers, data='{"format":"csv"}')
progressID=json.loads(response.text)['result']['progressId'] # Get the ID for the download




####### Check if export complete
url = "https://eu.qualtrics.com/API/v3/surveys/{}/export-responses/{}".format(surveyid,progressID)
headers = {"content-type": "application/json",
        "X-API-TOKEN": mytoken
        }
    
export_complete=False  #Assume not complete
while export_complete==False:
    response = requests.request("GET", url, headers=headers, data='{"format":"csv"}')
    response_dict=json.loads(response.text)['result']
    
    if float(response_dict['percentComplete'])>=100: #If complete, update value to break loop
        export_complete=True
        break
    time.sleep(2) #Wait 1 second before restarting loop if not
    

fileId=response_dict['fileId'] #Retrieve the file ID of the download    

####### Check if export complete

url = "https://eu.qualtrics.com/API/v3/surveys/{}/export-responses/{}/file".format(surveyid,fileId)
headers = {"content-type": "application/json",
        "X-API-TOKEN": mytoken
        }
response = requests.request("GET", url, headers=headers, stream=True)

In [ ]:

# Step 4: Unzipping the file and storing it locally
zipfile.ZipFile(io.BytesIO(response.content)).extractall("Leavers Survey")
print('Complete')

 



### Long term goals

1) Have a single file combining the four surveys, all time.

2) Dynamically look up the last response for one of the surveys, and pull data from then until now, or have option to pull a specified date range, or a relative date range to now.

3) Pull the latest data for each of the 4 surveys

4) Pull the question spec: Match question text, conditionally format based on type

5) Append the data to history for each of the surveys

6) Union dynamically to create a master file with appropriate fields overlapping
